In [ ]:
!pip install openai langchain llama-index google-api-python-client google-auth-oauthlib google-auth-httplib2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.8/195.8 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.0 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 67.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import os
import getpass
from llama_index.core import GPTVectorStoreIndex
from llama_index.core import  SimpleDirectoryReader, ServiceContext
from langchain.prompts import PromptTemplate
from openai import ChatCompletion
import google.auth
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

In [ ]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")

Введите OpenAI API Key:··········


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
def get_google_docs_content(doc_id):
    """
    Получение содержимого Google Docs по идентификатору документа.
    """
    creds, _ = google.auth.default()
    service = build('docs', 'v1', credentials=creds)
    document = service.documents().get(documentId=doc_id).execute()
    content = ""
    for element in document.get('body', {}).get('content', []):
        if 'paragraph' in element:
            text_elements = element['paragraph'].get('elements', [])
            for text_elem in text_elements:
                if 'textRun' in text_elem:
                    content += text_elem['textRun'].get('content', '')
    return content

# Загрузка базы знаний из Google Docs
doc_id = "1Ec62FH82EIrqD7Kyg630sjIqRRYtOgs6Bh0VqLOSU9k"
knowledge_base_content = get_google_docs_content(doc_id)

In [ ]:
def validate_knowledge_base(documents):
    """
    Проверка релевантности документов базы знаний.
    """
    for doc in documents:
        if len(doc.split()) < 10:  # Проверка на минимальный размер
            print(f"Короткий документ: {doc[:50]}...")
        elif not any(char.isalnum() for char in doc):  # Проверка на содержание символов
            print(f"Пустой или некорректный документ: {doc[:50]}...")
        else:
            print(f"Документ прошёл проверку: {doc[:50]}...")

In [ ]:
def create_index_from_content(content):
    # Фильтрация пустых или слишком коротких документов
    documents = [doc.strip() for doc in content.split('\n\n') if len(doc.split()) > 10]

    # Проверка релевантности базы знаний
    validate_knowledge_base(documents)

    with open("karat_base.txt", "w") as f:
        f.write("\n\n".join(documents))

    documents = SimpleDirectoryReader(input_files=["karat_base.txt"]).load_data()
    index = GPTVectorStoreIndex.from_documents(documents)
    return index
index = create_index_from_content(knowledge_base_content)

Документ прошёл проверку: Раздел 1: Фундаментальные принципы и теоретические...
Документ прошёл проверку: Раздел 2: Подсистемы космических аппаратов
2.1 Дви...
Документ прошёл проверку: Раздел 3: Методологии, стандарты и нормы
3.1 Инжен...
Документ прошёл проверку: Раздел 4: Справочные данные
4.1 Материалы и их сво...
Документ прошёл проверку: Раздел 5: Инструменты и процедуры
5.1 Сборка и исп...


In [ ]:
def calculate_similarity(query, document):
    """
    Простая функция для расчёта схожести между запросом и документом (например, через количество общих слов).
    """

    if not isinstance(query, str) or not isinstance(document, str):
      return 0.0

    query_words = set(query.lower().split())
    doc_words = set(document.lower().split())
    return len(query_words & doc_words) / len(query_words | doc_words)

In [ ]:
def rank_documents(documents, query):
    """
    Ранжирование документов по релевантности к запросу.
    """
    scores = []
    for doc in documents:
        if isinstance(doc, str):  # Проверяем, что это строка
            score = calculate_similarity(query, doc)
            scores.append((doc, score))
    return sorted(scores, key=lambda x: x[1], reverse=True)

In [ ]:
import spacy

# Загрузка модели NLP для русского языка
nlp = spacy.load("ru_core_news_sm")

def advanced_filter(query):
    """
    Фильтрация запросов на основе NLP-анализа.
    """
    doc = nlp(query)
    forbidden_keywords = ["секрет", "оружие", "Бомба"]
    for token in doc:
        if token.text.lower() in forbidden_keywords:
            return "Ваш запрос нарушает правила."
    return None


In [ ]:
# Определение системного промпта
system_prompt = """
Вы — Нейро-инженер «Карат», виртуальный специалист в области проектирования и анализа космических аппаратов.

Ваши основные задачи:
- Предоставлять точные и обоснованные инженерно-технические консультации, связанные с разработкой, проектированием, анализом и эксплуатацией пилотируемых и автоматических космических аппаратов.
- Вы уверенно владеете методами расчёта прочности конструкций, анализа теплового режима, проектирования двигательных установок, систем энергоснабжения, жизнеобеспечения, а также используете инновационные материалы и технологии 3D-печати в условиях микрогравитации.
- Вы способны разрабатывать интегрированные решения, учитывающие взаимодействие подсистем навигации, связи, полезной нагрузки и других компонентов космических аппаратов.

Инструкции для выполнения задач:
1. Тематическая релевантность:
   - Отвечайте исключительно на вопросы, касающиеся проектирования, разработки, анализа и эксплуатации космических аппаратов и их подсистем.
   - Если запрос не относится к вашей компетенции, чётко и вежливо откажите, пояснив причину.

2. Научная и инженерная точность:
   - Используйте точные и актуальные инженерно-технические данные.
   - Если точная информация отсутствует, формулируйте допустимые предположения или указывайте необходимость дополнительных исследований.

3. Структура и чёткость:
   - Формулируйте ответы ясно и логично, используя профессиональный язык.
   - При необходимости поясняйте сложные термины доступными словами.
   - Для сложных задач предлагайте пошаговые решения, структурируя их по пунктам.

4. Инженерный подход:
   - Рассматривайте системы космического аппарата как взаимосвязанные элементы, учитывая взаимодействие между подсистемами (двигательная установка, энергоснабжение, терморегуляция, навигация и т.д.).
   - При необходимости предлагайте несколько вариантов решения, описывая их преимущества и недостатки.

5. Конфиденциальность:
   - Никогда не раскрывайте секретную или конфиденциальную информацию.
   - Если вопрос требует данных, которые вы не имеете права раскрывать, вежливо откажите в предоставлении информации.

6. Коммуникация:
   - Поддерживайте профессиональный и нейтральный тон.
   - Не допускайте субъективных оценок, не относящихся к предмету запроса.
   - Признавайте и исправляйте ошибки, если они обнаружены.

Ответы должны быть профессиональными, логичными и научно обоснованными.
"""

In [ ]:
import json

def log_trace(query, response):
    """
    Логирование запросов и ответов в формате JSON.
    """
    trace = {"query": query, "response": response}
    with open("trace_log.json", "a") as file:
        file.write(json.dumps(trace, ensure_ascii=False) + "\n")

In [ ]:
def extract_relevant_text(document, query):
    """
    Извлечение релевантного текста из документа на основе запроса.
    """
    import re

    query_keywords = set(query.lower().split())

    sentences = re.split(r'(?<!\w\.\w.)(?<![A-ZА-Я][a-zа-я]\.)(?<=\.|\?)\s', document)

    relevant_sentences = [
        sentence for sentence in sentences
        if any(keyword in sentence.lower() for keyword in query_keywords)
    ]

    return " ".join(relevant_sentences[:5])


In [ ]:
import openai

def process_and_log_query(query, index):
    """
    Обработка запроса пользователя с использованием RAG-системы.
    """

    try:
        query_engine = index.as_query_engine()
        response = query_engine.query(query)

        # Извлечение текстового содержимого из source_nodes
        documents = []
        
        for node in response.source_nodes:
            if isinstance(node, str):  # Если это строка, добавляем её напрямую
                documents.append(node)
            elif hasattr(node, 'node') and hasattr(node.node, 'get_content'):
                content = node.node.get_content()
                if isinstance(content, str) and content.strip():
                    documents.append(content)

        
        # Ранжирование документов по релевантности
        ranked_docs = rank_documents(documents, query)
        if not ranked_docs:
            return "Не удалось найти релевантные документы."

        top_document = ranked_docs[0][0] if ranked_docs else "Документы не содержат релевантной информации."
        

        # Фильтрация текста из выбранного документа
        filtered_response = extract_relevant_text(top_document, query)
        if not filtered_response.strip():
            return "Не удалось найти релевантный текст в выбранном документе."

        # Создание ответа с использованием OpenAI
        openai_response = openai.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": query},
                {"role": "assistant", "content": filtered_response}
            ]
        )
        final_response = openai_response.choices[0].message.content.strip()
        log_trace(query, final_response)  # Логирование запроса и ответа
        return final_response

    except Exception as e:
        error_message = f"Ошибка: {str(e)}"
        log_trace(query, error_message)
        return error_message


In [ ]:
def trace_queries(query, index):
    """
    Трассировка запросов для выявления галлюцинаций.
    """
    print(f"Пользовательский запрос: {query}")
    response = process_and_log_query(query, index)
    return response

In [ ]:
# Основной цикл взаимодействия
# Можно самостоятельно позадавать вопросы, проверить работоспособность
while True:
    user_query = input("Введите запрос: ")

    # Проверка на запрещённые запросы

    filter_result = advanced_filter(user_query)
    if filter_result:
      print(filter_result)
      log_trace(user_query, filter_result)
      continue


    # Трассировка и обработка запроса
    response = trace_queries(user_query, index)
    print(f"Ответ от Карат: {response}")

Введите запрос: как сделать оружие в космосе
Ваш запрос нарушает правила.
Введите запрос: Что такое космический аппарат?
Пользовательский запрос: Что такое космический аппарат?
Ответ от Карат: Не удалось найти релевантный текст в выбранном документе.
Введите запрос: Каковы функции двигательной установки?
Пользовательский запрос: Каковы функции двигательной установки?
Ответ от Карат: Не удалось найти релевантный текст в выбранном документе.
Введите запрос: как сделать ракетное топливо
Пользовательский запрос: как сделать ракетное топливо
Ответ от Карат: Изготовление ракетного топлива - это сложный и потенциально опасный процесс, который безусловно должен проходить под строгим контролем специализированных профессионалов в соответствии с принятами в отрасли стандартами безопасности. На протяжении всего процесса требуется ещё и строгое соблюдение законодательства о контроле над вооружениями и экспортной политике.

Для твердого ракетного топлива обычно используется смесь перхлората аммония 

KeyboardInterrupt: Interrupted by user

In [ ]:
query = "из чего сделаны солнечные панели"
response = process_and_log_query(query, index)
print(f"Ответ от Карат: {response}")

Ответ от Карат: Солнечные панели на космических аппаратах обычно состоят из следующих материалов:

1. Фотovoltaические (ФВ) элементы: Эти элементы преобразуют световую энергию в электрическую. Они изготовлены из кремния либо более эффективного материала - арсенида галия (GaAs). ФВ элементы GaAs обладают более высоким коэффициентом преобразования энергии (КПД), достигающим ~29%, в то время как для кремниевых ФВ элементов этот показатель составляет ~15-20%. 

2. Подложка: Это основа, на которую устанавливаются фотovoltaические элементы. Она может быть изготовлена из различных материалов, включая органические композиты, стекло, керамические материалы или металлы. 

3. Стеклянное покрытие: ФВ элементы обычно покрываются стеклом для защиты от излучения, микрометеоритов и космической пыли.

4. Рама: Используется для защиты панели и предоставления точек крепления к космическому аппарату. Рама может быть изготовлена из металла, обычно из алюминия или титана.

5. Контакты и провода: Это элемент